## Sedaro Validaton Notebook
### Attitude Dynamics

Compare to basilisk.

### Reproducing our Results

To ensure reproducibility, the directory containing this notebook also includes a `requirements.txt` file that specifies the exact package versions that were used. To create a similar environment, use the following sequence of commands with Python `3.11` and the built-in `venv` package. See the [venv documentation](https://docs.python.org/3/library/venv.html) for more details on how this works.

- In a unix-like terminal:

    ```zsh
    > python -m venv .venv
    > source .venv/bin/activate
    > pip install -r requirements.txt
    ```

- In a Windows `cmd.exe` terminal:
    ```bat
    C:\> python -m venv .venv
    C:\> .venv\Scripts\activate.bat
    C:\> pip install -r requirements.txt
    ```

- In Windows PowerShell:

    ```bat
    C:\> python -m venv .venv
    C:\> .venv\Scripts\Activate.ps1
    C:\> pip install -r requirements.txt
    ```

Confirm that the Jupyter notebook is using this virtual evironment before running the code below.

In [265]:
# FIXME: Use pip freeze > requirements.txt when the notebook is finalized.
#        Ensure that there are no unnecessary packages or local installations.
import json

import matplotlib.pyplot as plt
import pandas
import numpy as np
from sedaro import SedaroApiClient
from utils import mrp_to_quaternion, angleBetweenClosestQuaternions, findClosestIndex, angleBetweenQuaternion

### Important: Read Before Running

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [266]:
with open('/Users/richard/sedaro/satellite-app/secrets.json', 'r') as file:
    API_KEY = json.load(file)['garfunkel']
# with open('../../secrets.json', 'r') as file: FIXME
#     API_KEY = json.load(file)['API_KEY_LOCAL']

In [268]:
SCENARIO_BRANCH_ID = 'PKnnzr5ZZDs2mKTfqXnpqs'  # FIXME: Populate this for them, since it is static
VEHICLE_BRANCH_ID = 'PKnnwtdWXcdfCDk9LzZb8T'

sedaro = SedaroApiClient(API_KEY, host='http://localhost')
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

## (Optional) Edit Scenario to Create Agents with Varying Orbits

In [242]:
stations = pandas.read_csv('stations.csv')
kinematics = scenario.EcefStationaryKinematics.get_first()
_ = [p.delete() for p in scenario.PeripheralGroundPoint.get_all()]
# Helper fn
def get_lat(lat_str:str) -> float:
    s = lat_str.split('°')
    if s[1] == ' N':
        return float(s[0])
    elif s[1] == ' S':
        return -float(s[0])
    else:
        raise ValueError(f'Unknown hemisphere {s[1]}')
def get_lon(lon_str:str) -> float:
    s = lon_str.split('°')
    if s[1] == ' E':
        return float(s[0])
    elif s[1] == ' W':
        return -float(s[0])
    else:
        raise ValueError(f'Unknown hemisphere {s[1]}')

# Add stations as agent and as tg
crud = []
names = []
for _, row in stations.iterrows():
    name = row['Train Station']
    if name in names:
        name = row['City'] + ' ' + name
    names.append(name)
    crud.append({
        'name': name[:32],
        'type': 'PeripheralGroundPoint',
        'lat': {'deg': get_lat(row['Latitude'])},
        'lon': {'deg': get_lon(row['Longitude'])},
        'alt': {'km': 0},
        'id': f'${name}',
        'kinematics': kinematics.id,
    })
crud.append({
    'name': 'Some Train Stations',
    'type': 'AgentGroup',
    'agentType': 'GroundTarget',
    'agentAssociations': {f'${name}': {'priority': i} for i, name in enumerate(names)},
})
_ = scenario.crud(blocks=crud)

In [246]:
vehicle = sedaro.agent_template(VEHICLE_BRANCH_ID)
vehicle.TargetGroup.get_first().update(sortValue='ELEVATION', sortOrder='DESCENDING')


TargetGroup(
   activeTarget=None
   disabled=False
   filterConditions=[]
   id='PKnnxky8XfpS4kxCTtnxNh'
   name='Point at These'
   sortOrder='DESCENDING'
   sortValue='ELEVATION'
   targetAssociations={}
   targetType='GroundTarget'
   targets=[]
   type='TargetGroup'
)

In [277]:
base_agent = scenario.TemplatedAgent.get_first()
blocks_to_delete = []
for a in scenario.TemplatedAgent.get_all():
    if a.id == base_agent.id:
        continue
    blocks_to_delete.append(a.id)
    blocks_to_delete.append(a.kinematics.id)
scenario.crud(delete=blocks_to_delete)
base_orbit = base_agent.kinematics
orbit_params = base_orbit.initialStateDefParams

for i, inc in enumerate(np.linspace(0, 170, 10)):
    new_orbit = base_orbit.clone()
    new_orbit = new_orbit.update(initialStateDefParams={
        **orbit_params,
        'inc': inc,
        'raan': 90,
        # 'nu': 2*(np.abs(inc)-90)+90,
        # 'raan': 180.
    })
    new_agent = base_agent.clone().update(kinematics=new_orbit.id, name=f'Agent {i}')

In [275]:
scenario.simulation.start()

In [269]:
for i, v in enumerate(scenario.PeripheralGroundPoint.get_all()):
    if i % 4 == 0:
        continue
    v.delete()

## Download Scenario Data

This notebook considers the following reference scenario(s):
- [Scenario Name 1](https://satellite.sedaro.com/shareable_link)
- [Scenario Name 2](https://satellite.sedaro.com/shareable_link)

In [ ]:
results = sedaro.scenario(SCENARIO_BRANCH_ID).simulation.results()
agent_results = results.agent(results.templated_agents[0])

In [ ]:
template_branch = 'PKgktBPKwNRrQh2KtsTjtc'
vehicle = sedaro.agent_template(template_branch)
wheels = vehicle.ReactionWheel.get_all()
x_wheel_id = next(wheel.id for wheel in wheels if 'X' in wheel.name)
y_wheel_id = next(wheel.id for wheel in wheels if 'Y' in wheel.name)
z_wheel_id = next(wheel.id for wheel in wheels if 'Z' in wheel.name)

## Prepare Data

In [ ]:
sedaro_attitude_results = np.array(agent_results.block('root').attitude.body_eci.values)
sedaro_ts = agent_results.block('root').attitude.body_eci.elapsed_time
sedaro_commanded = np.array([v if v else [0, 0, 0, 0] for v in agent_results.block('root').commandedAttitude.values])

Read basilisk results

In [ ]:
with open('sedaro_rw_basilisk_lock.csv', 'r') as f:
    basilisk_results = pandas.read_csv(f)

In [ ]:
basilisk_results['attitude_q'] = [mrp_to_quaternion((x,y,z)) for x,y,z in zip(basilisk_results['attitude_mrp_x'], basilisk_results['attitude_mrp_y'], basilisk_results['attitude_mrp_z'])]
basilisk_time_arr = basilisk_results['time'].to_numpy()

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

basilisk_attitude = np.array(basilisk_results['attitude_q'].to_list())


fig, ax = plt.subplots(3, 1)
# Plot x result
ax[0].plot(basilisk_results['time'], basilisk_attitude[:,0], label='basilisk')
ax[0].plot(sedaro_ts, sedaro_attitude_results[:,0], label='sedaro')
ax[0].plot(sedaro_ts, -sedaro_attitude_results[:, 0], ':', label='-sedaro')
ax[0].legend()
ax[0].set_ylabel('Quaternion x')
# ax[0].legend()
# Plot y result, no legend
ax[1].plot(basilisk_results['time'], basilisk_attitude[:,1])
ax[1].plot(sedaro_ts, sedaro_attitude_results[:,1])
ax[1].plot(sedaro_ts, -sedaro_attitude_results[:, 1], ':')
ax[1].set_ylabel('Quaternion y')
# Plot z result, no legend
ax[2].plot(basilisk_results['time'], basilisk_attitude[:,2])
ax[2].plot(sedaro_ts, sedaro_attitude_results[:,2])
ax[2].plot(sedaro_ts, -sedaro_attitude_results[:, 2], ':')
ax[2].set_ylabel('Quaternion z')
ax[2].set_xlabel('Time (s)')
fig.set_tight_layout(True)
plt.show()

In [ ]:
# Plot the angle between
fig, ax = plt.subplots(1, 1)
basilisk_time_arr = basilisk_results['time'].to_numpy()
diff_angles = [np.degrees(angleBetweenClosestQuaternions(sedaro_q, t_1, basilisk_attitude, basilisk_results['time'])) for sedaro_q, t_1 in zip(sedaro_attitude_results, sedaro_ts)]
ax.plot(sedaro_ts, diff_angles)
ax.set_ylabel('Angle between closest quaternions (deg)')
ax.set_xlabel('Time (s)')

In [ ]:
sedaro_gg = np.array(agent_results.block('root').gravityGradientTorque.values)

In [ ]:
basilisk_gg = np.array([[x,y,z] for x,y,z in zip(basilisk_results['gg_torque_x'], basilisk_results['gg_torque_y'], basilisk_results['gg_torque_z'])])

In [ ]:
indices = [findClosestIndex(s_t, basilisk_time_arr) for s_t in sedaro_ts]
diff_torque = [np.linalg.norm(s_gg - basilisk_gg[i]) for i, s_gg in zip(indices, sedaro_gg)]
diff_angles = [angleBetweenQuaternion(s_q, basilisk_attitude[i]) for i, s_q in zip(indices, sedaro_attitude_results)]
fig, ax = plt.subplots()
ax.scatter(diff_angles, diff_torque)
ax.set_xlabel('Angle between quaternions')
ax.set_ylabel('Difference in torque')

In [ ]:
diff_torque_v = np.array([s_gg - basilisk_gg[i] for i, s_gg in zip(indices, sedaro_gg)])
fig, ax = plt.subplots()
ax.plot(sedaro_ts, diff_torque_v[:, 0], label='x')
ax.plot(sedaro_ts, diff_torque_v[:, 1], label='y')
ax.plot(sedaro_ts, diff_torque_v[:, 2], label='z')
ax.legend()
ax.set_xlabel('Time (s)')
ax.set_ylabel('Torque Difference (Nm)')

In [ ]:
fig,ax = plt.subplots(3, 1)
ax[0].plot(sedaro_ts, agent_results.block(x_wheel_id).speed.values, label='sedaro')
ax[0].plot(basilisk_time_arr, basilisk_results['rw_x_omega'], label='basilisk')
ax[0].set_ylabel('Wheel Speed x')
ax[0].legend()
ax[1].plot(sedaro_ts, agent_results.block(y_wheel_id).speed.values)
ax[1].plot(basilisk_time_arr, basilisk_results['rw_y_omega'])
ax[1].set_ylabel('Wheel Speed y')
ax[2].plot(sedaro_ts, agent_results.block(z_wheel_id).speed.values)
ax[2].plot(basilisk_time_arr, basilisk_results['rw_z_omega'])
ax[2].set_ylabel('Wheel Speed z')
ax[2].set_xlabel('Time (s)')
fig.set_tight_layout(True)
plt.show()